In [1]:
import pandas as pd
import csv
import re

In [2]:
# names of files to read from
r_maxo_classes_with_definitionsTSV = '~/Git/MAxO/src/ontology/sparql-test/maxo_classes_with_definitions.tsv'
r_ncit_definitionsTSV = '~/Git/MAxO/src/ontology/sparql-test/ncit_definitions.tsv'

tsv_read_maxo = pd.read_csv(r_maxo_classes_with_definitionsTSV, sep='\t')
tsv_read_ncit = pd.read_csv(r_ncit_definitionsTSV, sep='\t')

maxo_id=list()
ncit_id=list()


In [3]:
tsv_read_maxo.columns

Index(['?cls', '?xref', '?def'], dtype='object')

In [4]:
from pandas import DataFrame

x=()
mylist=[]
newlist=list()
#extract class IRI, definition, and xref from maxo file
maxo_id = pd.DataFrame(tsv_read_maxo)
cols = ["?cls","?xref","?def"]
#extract MAXO ID from class IRI
maxo_id = maxo_id[maxo_id.columns[0]]
for line in maxo_id:
    line=line.strip('/')
    x=re.findall('[A-Z]{4,11}_[A-Z0-9]{1,15}', line)
    x=[item.replace('_', ':') for item in x]
    mylist.append(x)
    maxo_df= DataFrame(mylist,columns=['Maxo_ID'])
#join maxo ID to the file
maxo_id_def= maxo_df.join(tsv_read_maxo, lsuffix="_left", rsuffix="_right")
print(maxo_id_def.head(2))
    
with open("maxo_xref_definitions.tsv",'wb') as out:
    maxo_id_def.to_csv('maxo_xref_definitions.tsv', encoding='utf-8', sep='\t', index=False)
    
    

        Maxo_ID                                           ?cls         ?xref  \
0  MAXO:0000455  <http://purl.obolibrary.org/obo/MAXO_0000455>   NCIT:C15215   
1  MAXO:0000454  <http://purl.obolibrary.org/obo/MAXO_0000454>  NCIT:C111157   

                                                ?def  
0  The use of extreme cold to damage or destroy a...  
1  Removal, separation, detachment, extirpation, ...  


In [5]:
y=()
newlist=[]
#extract just the ncit_id
ncit_id = pd.DataFrame(tsv_read_ncit)
cols = ["?cls","?def"]
ncit_id = ncit_id[ncit_id.columns[0]]

for line in ncit_id:
    line=line.strip('/')
    y=re.findall('[A-Z]{4,11}_[A-Z0-9]{1,15}', line)
    y=[item.replace('_', ':') for item in y]
    newlist.append(y)
    ncit_df= DataFrame(newlist,columns=['NCIT_ID'])
ncit_id_def= ncit_df.join(tsv_read_ncit, lsuffix="_left", rsuffix="_right")
print(ncit_id_def.head(2))
    
with open("ncit_definitions.tsv",'wb') as out:
    ncit_id_def.to_csv('ncit_definitions.tsv', encoding='utf-8', sep='\t', index=False)
    

        NCIT_ID                                           ?cls  \
0  NCIT:C104459  <http://purl.obolibrary.org/obo/NCIT_C104459>   
1  NCIT:C113621  <http://purl.obolibrary.org/obo/NCIT_C113621>   

                                                ?def  
0           A protein encoded by the human BID gene.  
1  H/ACA ribonucleoprotein complex subunit 4 (514...  


ncit_id_def.info

In [6]:

maxo_id_def.columns = ["Maxo_ID","?cls","ID","?def"]
print(maxo_id_def.head())

        Maxo_ID                                           ?cls  \
0  MAXO:0000455  <http://purl.obolibrary.org/obo/MAXO_0000455>   
1  MAXO:0000454  <http://purl.obolibrary.org/obo/MAXO_0000454>   
2  MAXO:0000714  <http://purl.obolibrary.org/obo/MAXO_0000714>   
3  MAXO:0000196  <http://purl.obolibrary.org/obo/MAXO_0000196>   
4  MAXO:0000300  <http://purl.obolibrary.org/obo/MAXO_0000300>   

                                       ID  \
0                             NCIT:C15215   
1                            NCIT:C111157   
2                                abms.org   
3  <http://orcid.org/0000-0001-7941-2961>   
4                           PMID:21857074   

                                                ?def  
0  The use of extreme cold to damage or destroy a...  
1  Removal, separation, detachment, extirpation, ...  
2  Evaluation by a physician who specializes in t...  
3  An antibacterial substance with aminoglycoside...  
4  The drug treatment of inflammatory bowel disea...  


In [6]:
maxo_id_list= []
maxo_def_list= []
maxo_def_xref_list= []
ncit_id_list=[]
ncit_def_list= []
#find MAXO definitions ONLY with NCIT xrefs
for index, row in maxo_id_def.iterrows():
    if row[2].startswith("NCIT:"):
        for index, rows in ncit_id_def.iterrows():
            #determine if the the MAXO def xref matches the NCIT ID
            if row[2] == rows[0]:
                maxo_id_list.append(row[0])
                maxo_def_list.append(row[3])
                maxo_def_xref_list.append(row[2])
                ncit_def_list.append(rows[2])
                ncit_id_list.append(rows[0])
            else:
                continue


In [8]:

maxo_ncit_def_df=pd.DataFrame(list(zip(maxo_id_list, maxo_def_list, maxo_def_xref_list, ncit_def_list, ncit_id_list,)), columns=["maxo_id","maxo_def", "maxo_def_xref", "ncit_def", "ncit_id"])
print(maxo_ncit_def_df.head())

with open("maxo_ncit_def.tsv",'wb') as out:
    maxo_ncit_def_df.to_csv('maxo_ncit_def.tsv', encoding='utf-8', sep='\t', index=False)
    


        maxo_id                                           maxo_def  \
0  MAXO:0000455  The use of extreme cold to damage or destroy a...   
1  MAXO:0000454  Removal, separation, detachment, extirpation, ...   
2  MAXO:0000838  The determination of the ratio of the prothrom...   
3  MAXO:0000146  A procedure to determine the number of cells i...   
4  MAXO:0000728  A technique in which sound waves are sent out ...   

  maxo_def_xref                                           ncit_def  \
0   NCIT:C15215  The use of extreme cold to damage or destroy a...   
1  NCIT:C111157  Removal, separation, detachment, extirpation, ...   
2  NCIT:C170591  The determination of the ratio of the prothrom...   
3   NCIT:C48938  A procedure to determine the number of cells i...   
4   NCIT:C17644  A technique in which sound waves are sent out ...   

        ncit_id  
0   NCIT:C15215  
1  NCIT:C111157  
2  NCIT:C170591  
3   NCIT:C48938  
4   NCIT:C17644  
